In [1]:
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Flatten
from matplotlib import pyplot
from PIL import Image
import os, sys
from sklearn.model_selection import train_test_split
import numpy

In [2]:
# Define the encoder block based on the original paper
def define_encoder_block(layer, filtersNo, batchnorm=True):
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # in the original paper, all convolution kernels are (4,4), with stride 2. Stride for decoder means downsampling.
    x = Conv2D(filtersNo, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)
    
    # Conditional batch normalization (important for the first layer)
    if batchnorm:
        x = BatchNormalization()(x, training=True)
        
    # All ReLUs in the encoder are leaky!
    x = LeakyReLU(alpha=0.2)(x)
    
    return x

In [3]:
# Define the decoder block based on the original paper
def decoder_block(layer, skip, filtersNo, dropout=True, batch=True):
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # in the original paper, all convolution kernels are (4,4), with stride 2. Stride for decoder means upsampling.
    x = Conv2DTranspose(filtersNo, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer)

    # All layers in the original paper have batch normalization, although we set an if statement just to play with the model
    if batch:
        x = BatchNormalization()(x, training=True)
        
    # Some decoder layers don't have dropout
    if dropout:
        x = Dropout(0.5)(x, training=True)
        
    # Merge with skip connection
    x = Concatenate()([x, skip])
    
    # All ReLUs in the decoder are not leaky!
    x = Activation('relu')(x)
    
    return x

In [16]:
# Define the generator based on encoder/decoder
def define_generator():
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # image input
    inputImage = Input(shape=(128,128,1))
    
    ###### Encoder
    
    # C64, input (128,128,1), output (64,64,64)
    encoderLayer1 = define_encoder_block(inputImage, 64, batchnorm=False)
    
    #C128, input (64,64,64), output (32,32,128)
    encoderLayer2 = define_encoder_block(encoderLayer1, 128)
    
    #C256, input (32,32,128), output (16,16,256)
    encoderLayer3 = define_encoder_block(encoderLayer2, 256)
    
    #C512, input (16,16,256), output (8,8,512)
    encoderLayer4 = define_encoder_block(encoderLayer3, 512)
    
    #C512, input (8,8,512), output (4,4,512)
    encoderLayer5 = define_encoder_block(encoderLayer4, 512)
    
    #C512, input (4,4,512), output (2,2,512)
    encoderLayer6 = define_encoder_block(encoderLayer5, 512)
    
    ###### Bottleneck layer, will have an input of (2,2,512) and an output of (1,1,512)
    bottleneck = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(encoderLayer6)
    bottleneck = Activation('relu')(bottleneck)
    
    ###### Decoder, with skip connection
    
    #CD512
    decoderLayer1 = decoder_block(bottleneck, encoderLayer6, 512)
    
    #CD512
    decoderLayer2 = decoder_block(decoderLayer1, encoderLayer5, 512)
    
    #C512
    decoderLayer3 = decoder_block(decoderLayer2, encoderLayer4, 512, dropout=False)
    
    #C256
    decoderLayer4 = decoder_block(decoderLayer3, encoderLayer3, 256, dropout=False)
    
    #C128
    decoderLayer5 = decoder_block(decoderLayer4, encoderLayer2, 128, dropout=False)
    
    #C64
    decoderLayer6 = decoder_block(decoderLayer5, encoderLayer1, 64, dropout=False)
    
    # Output with tanh function, as mentioned in the original paper. Output will be (128x128x3)
    g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(decoderLayer6)
    outputImage = Activation('tanh')(g)
    
    # Define model
    model = Model(inputImage, outputImage)
    
    return model

In [5]:
# Define the 70x70 discriminator as in the original paper
def define_discriminator():
    
    # init weights from a Gaussian distribution with mean 0 and standard deviation 0.02
    init = RandomNormal(stddev=0.02)
    
    # source image input
    source = Input(shape=(128,128,1))
    
    # target image input
    target = Input(shape=(128,128,3))
    
    # concatenate images channel-wise
    merged = Concatenate()([source, target])
    
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    
    # patch output
    d = Conv2D(1, (8,8), strides=(8,8), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    patch_out = Flatten()(patch_out)
    
    # define model
    model = Model([source, target], patch_out)
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [6]:
# Define Pix2Pix GAN
def pix2pix(generator, discriminator):
    
    # make weights in the discriminator not trainable
    for layer in discriminator.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
            
    # define the source image
    source = Input(shape=(128,128,1))
    
    # connect the source image to the generator input
    genOut = generator(source)
    
    # connect the source input and generator output to the discriminator input
    disOut = discriminator([source, genOut])
    
    # src image as input, generated image and classification output
    model = Model(source, [disOut, genOut])
    
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    
    return model

In [7]:
def generateFromDataset(trainX, trainY, samples):
    
    # Choose random images from both input and output
    no = randint(0, trainX.shape[0], samples)
    gx, gy = trainX[no], trainY[no]
    
    # Set y-labels to 1, as these images are from dataset
    y = ones((samples, 1))
    return [gx, gy], y

In [8]:
def generateFromGenerator(generator, samples):
    # Generate fake instance
    x = generator.predict(samples)
    
    # Labels will be zero because they come from the generator
    y = zeros((len(x), 1))
    
    return x, y

In [9]:
def summarize_performance(step, g_model, trainX, trainY, n_samples=3):
    # select a sample of input images
    [X_realA, X_realB], _ = generateFromDataset(trainX, trainY, n_samples)
    
    # generate a batch of fake samples
    X_fakeB, _ = generateFromGenerator(g_model, X_realA)
    
    # scale all pixels from [-1,1] to [0,1]
    #X_realA = (X_realA + 1) / 2.0
    #X_realB = (X_realB + 1) / 2.0
    #X_fakeB = (X_fakeB + 1) / 2.0
    
    # plot real source images
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realA[i])
    # plot generated target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(X_fakeB[i])
    # plot real target image
    for i in range(n_samples):
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(X_realB[i])
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [23]:
def train(discriminator, generator, gan, epochs=100000, samplesPerEpoch=250):
    
    # Load the data
    bwData = load("Flickr8kblackandwhite1dim.npy")
    colorData = load("flickr8k_shuffled.npy")
    y = numpy.ones((8091,1))
    
    BWSplit = numpy.array_split(bwData, 2)
    colorSplit = numpy.array_split(colorData, 2)
    
    # manually enumerate epochs
    for i in range(epochs):
        
        # Generate real samples
        [realX, realY], realLabel = generateFromDataset(BWSplit[0], colorSplit[0], samplesPerEpoch)
        
        # Generate fake samples
        fakeY, fakeLabel = generateFromGenerator(generator, realX)
        
        # Update discriminator on real samples
        realLoss = discriminator.train_on_batch([realX, realY], realLabel)
        
        # Update discriminator on fake samples
        fakeLoss = discriminator.train_on_batch([realX, fakeY], fakeLabel)
        
        # Update generator
        generatorLoss, _, _ = gan.train_on_batch(realX, [realLabel, realY])
        
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, realLoss, fakeLoss, generatorLoss))
        if i*samplesPerEpoch % 2500 == 0:
            summarize_performance(i, generator, BWSplit[0], colorSplit[0])

In [21]:
d = define_discriminator()
g = define_generator()
p2p = pix2pix(g,d)

In [24]:
train(d,g,p2p)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>1, d1[0.006] d2[0.001] g[14.429]
>Saved: plot_000001.png and model_000001.h5
>2, d1[0.005] d2[0.008] g[13.714]
>3, d1[0.004] d2[0.007] g[10.899]
>4, d1[0.003] d2[0.011] g[10.149]
>5, d1[0.003] d2[0.010] g[10.146]
>6, d1[0.003] d2[0.030] g[9.923]
>7, d1[0.005] d2[0.022] g[9.728]
>8, d1[0.003] d2[0.008] g[9.675]
>9, d1[0.006] d2[0.001] g[9.487]
>10, d1[0.005] d2[0.015] g[9.695]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>11, d1[0.002] d2[0.088] g[9.441]
>Saved: plot_000011.png and model_000011.h5
>12, d1[0.006] d2[0.001] g[9.503]
>13, d1[0.006] d2[0.011] g[9.354]
>14, d1[0.011] d2[0.000] g[9.436]
>15, d1[0.007] d2[0.001] g[9.441]
>16, d1[0.006] d2[0.001] g[9.008]
>17, d1[0.004] d2[0.081] g[9.307]
>18, d1[0.008] d2[0.000] g[8.824]
>19, d1[0.015] d2[0.000] g[9.347]
>20, d1[0.005] d2[0.001] g[9.460]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


...

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>14061, d1[0.000] d2[0.000] g[9.172]
>Saved: plot_014061.png and model_014061.h5
>14062, d1[0.000] d2[0.000] g[8.967]
>14063, d1[0.000] d2[0.001] g[6.849]
>14064, d1[0.000] d2[0.000] g[8.154]
>14065, d1[0.000] d2[0.000] g[6.294]
>14066, d1[0.000] d2[0.000] g[4.133]
>14067, d1[0.000] d2[0.000] g[5.829]
>14068, d1[0.000] d2[0.001] g[3.564]
>14069, d1[0.000] d2[0.000] g[3.693]
>14070, d1[0.000] d2[0.000] g[4.074]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


>14071, d1[0.000] d2[0.000] g[9.242]
>Saved: plot_014071.png and model_014071.h5
>14072, d1[0.000] d2[0.000] g[9.237]
>14073, d1[0.000] d2[0.000] g[4.402]


KeyboardInterrupt: 